In [1]:
import cv2
import numpy as np

from keras.layers import Input
from yolov4.model import yolov4_body
from yolov4.decode import Decode

Using TensorFlow backend.


In [2]:
def get_class(classes_path):
    with open(classes_path, encoding='utf-8') as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [3]:
weights_path = 'model_data/weights/yolov4-plate_number_weights.h5'
anchors_path = 'model_data/anchors/yolov4_anchors.txt'
classes_path = 'model_data/classes/yolov4-plate_number.names'

class_names = get_class(classes_path)
anchors = get_anchors(anchors_path)
num_anchors = len(anchors)
num_classes = len(class_names)

input_size = (416, 416)

# 分数阈值和nms_iou阈值
conf_thresh = 0.6
nms_thresh = 0.45

yolov4_model = yolov4_body(Input(shape=input_size+(3,)), num_anchors//3, num_classes)
yolov4_model.load_weights(weights_path)

_decode = Decode(conf_thresh, nms_thresh, input_size, yolov4_model, class_names)

In [9]:
image = cv2.imread('image/1.jpg')

image, boxes, scores, classes = _decode.detect_image(image, True)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()